In [1]:
## virtual environment setup
%pip install conda
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df2 = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv')
df2.head(20)

,author,posted_on,rating,text,state
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",1,I used to love Comcast. Until all these consta...,MI
1,"Vera of Philadelphia, PA","Nov. 19, 2016",1,I'm so over Comcast! The worst internet provid...,PA
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",1,If I could give them a negative star or no sta...,CA
3,"Dennis of Manchester, NH","Nov. 16, 2016",1,I've had the worst experiences so far since in...,NH
4,"Ryan of Bellevue, WA","Nov. 14, 2016",1,Check your contract when you sign up for Comca...,WA
5,"Terri of Mobile, AL","Nov. 9, 2016",1,Thank God. I am changing to Dish. They gave me...,AL
6,"Kellie of Salt Lake City, UT","Nov. 9, 2016",1,I Have been a long time customer and only have...,UT
7,"Kathleen of New Haven, CT","Nov. 6, 2016",2,There is a malfunction on the DVR manager whic...,CT
8,"Shira of Bloomfield, NJ","Nov. 5, 2016",1,Charges overwhelming. Comcast service rep was ...,NJ
9,"Kristy of Alpharetta, GA","Nov. 2, 2016",1,"I have had cable, DISH, and U-verse, etc. in t...",GA


# Visualisierung der strukturierten Daten

In [3]:
# Topic-Modellierung mit LDA (Latent Dirichlet Allocation)
# Ziel: Jeder Beschwerde ein dominantes Thema zuweisen, um das häufigste Thema pro Bundesstaat zu ermitteln.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

NUM_TOPICS = 5  # Anzahl der Themen

# 1. Texte vorbereiten
texts = df2['text'].dropna().astype(str).tolist()
print(f"✓ {len(texts)} Beschwerdetexte geladen")

# 2. Texte vektorisieren (Bag-of-Words)
vectorizer = CountVectorizer(
    max_df=0.95,            # Wörter, die in >95% der Dokumente vorkommen, ignorieren
    min_df=2,               # Wörter, die in <2 Dokumenten vorkommen, ignorieren
    stop_words='english',   # Englische Stoppwörter entfernen
    max_features=5000       # Maximal 5000 Features
)
doc_term_matrix = vectorizer.fit_transform(texts)
print(f"✓ Document-Term-Matrix: {doc_term_matrix.shape}")

# 3. LDA-Modell trainieren
lda_model = LatentDirichletAllocation(
    n_components=NUM_TOPICS,
    random_state=42,
    max_iter=20,
    learning_method='online'
)
doc_topics = lda_model.fit_transform(doc_term_matrix)
print(f"✓ LDA-Modell trainiert mit {NUM_TOPICS} Themen")

# 4. Topic-Labels aus den Top-Wörtern ableiten
feature_names = vectorizer.get_feature_names_out()
topic_labels = {}

print(f"\n{'='*60}")
print("Erkannte Themen (Top-10 Wörter pro Thema):")
print(f"{'='*60}")

for topic_idx, topic in enumerate(lda_model.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    topic_labels[topic_idx] = ", ".join(top_words[:3])  # Kurzlabel aus den Top-3-Wörtern
    print(f"\nThema {topic_idx}: {topic_labels[topic_idx]}")
    print(f"  Alle Top-10: {', '.join(top_words)}")

# 5. Dominantes Thema pro Beschwerde zuweisen
df2_with_topics = df2.copy()
# Nur für Zeilen mit Text
valid_mask = df2['text'].notna()
df2_with_topics.loc[valid_mask, 'dominant_topic'] = np.argmax(doc_topics, axis=1)
df2_with_topics.loc[valid_mask, 'topic_label'] = df2_with_topics.loc[valid_mask, 'dominant_topic'].map(topic_labels)

print(f"\n✓ Dominantes Thema für {valid_mask.sum()} Beschwerden zugewiesen")
print(f"\nVerteilung der Themen:")
print(df2_with_topics['topic_label'].value_counts())

✓ 5626 Beschwerdetexte geladen
✓ Document-Term-Matrix: (5626, 5000)
✓ LDA-Modell trainiert mit 5 Themen

Erkannte Themen (Top-10 Wörter pro Thema):

Thema 0: called, comcast, said
  Alle Top-10: called, comcast, said, technician, tech, time, service, appointment, told, phone

Thema 1: month, comcast, 00
  Alle Top-10: month, comcast, 00, contract, 99, year, package, price, fee, pay

Thema 2: service, comcast, cable
  Alle Top-10: service, comcast, cable, customer, told, phone, internet, just, box, called

Thema 3: comcast, service, internet
  Alle Top-10: comcast, service, internet, speed, problem, tv, cable, time, channels, business

Thema 4: comcast, service, account
  Alle Top-10: comcast, service, account, told, called, credit, phone, customer, number, did

✓ Dominantes Thema für 5626 Beschwerden zugewiesen

Verteilung der Themen:
topic_label
service, comcast, cable       2598
comcast, service, account     1279
comcast, service, internet     764
called, comcast, said          620
m

In [5]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

print(f"Datensatz geladen: {len(df2)} Beschwerden")
print(f"Verfügbare Spalten: {df2.columns.tolist()}")

# Beschwerdeanzahlen pro Bundesstaat aggregieren
complaints_by_state = df2.groupby('state').size().reset_index(name='complaints')
complaints_by_state = complaints_by_state.sort_values('complaints', ascending=False)

# --- Überproportional dominantes Thema pro Bundesstaat ---
# Statt einfach das häufigste Thema zu nehmen (das national dominant ist),
# wird berechnet, welches Thema in einem Bundesstaat ÜBERREPRÄSENTIERT ist
# im Vergleich zur nationalen Verteilung.

# 1. Nationale Themenverteilung berechnen
national_topic_dist = df2_with_topics['dominant_topic'].value_counts(normalize=True)
print(f"\nNationale Themenverteilung:")
for topic, share in national_topic_dist.items():
    print(f"  Thema {int(topic)} ({topic_labels.get(int(topic), '?')}): {share:.1%}")

# 2. Themenverteilung pro Bundesstaat + Überrepräsentation berechnen
def get_distinctive_topic(state_group):
    """Ermittelt das Thema, das im Bundesstaat am stärksten überrepräsentiert ist."""
    state_dist = state_group['dominant_topic'].value_counts(normalize=True)
    
    best_topic = None
    best_ratio = -1
    
    for topic in state_dist.index:
        state_share = state_dist[topic]
        national_share = national_topic_dist.get(topic, 0.001)  # Fallback
        ratio = state_share / national_share  # Überrepräsentationsfaktor
        
        if ratio > best_ratio:
            best_ratio = ratio
            best_topic = topic
    
    return pd.Series({
        'dominant_topic': int(best_topic) if best_topic is not None else -1,
        'overrepresentation': best_ratio
    })

distinctive_topics = (
    df2_with_topics[df2_with_topics['dominant_topic'].notna()]
    .groupby('state')
    .apply(get_distinctive_topic)
    .reset_index()
)
distinctive_topics['topic_label'] = distinctive_topics['dominant_topic'].map(topic_labels)

# USA = 50 US-Bundesstaaten + DC
all_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC'
]

all_states_df = pd.DataFrame({'state': all_states})
all_states_complaints = all_states_df.merge(complaints_by_state, on='state', how='left')
all_states_complaints['complaints'] = all_states_complaints['complaints'].fillna(0).astype(int)

# Überproportional dominantes Thema pro Bundesstaat hinzufügen
all_states_complaints = all_states_complaints.merge(distinctive_topics, on='state', how='left')
all_states_complaints['topic_label'] = all_states_complaints['topic_label'].fillna('Keine Daten')
all_states_complaints['overrepresentation'] = all_states_complaints['overrepresentation'].fillna(0)

print(f"\nBeschwerdeanzahl + überproportional dominantes Thema pro Bundesstaat:")
print(all_states_complaints[['state', 'complaints', 'topic_label', 'overrepresentation']]
      .sort_values('complaints', ascending=False).to_string(index=False))

# Themenverteilung über Bundesstaaten
print(f"\nVerteilung der überproportionalen Themen über Bundesstaaten:")
print(all_states_complaints[all_states_complaints['complaints'] > 0]['topic_label'].value_counts())

print(f"\n✓ Karte erstellen mit {len(all_states_complaints)} Bundesstaaten")
print(f"  - mit Beschwerden:  {(all_states_complaints['complaints'] > 0).sum()}")
print(f"  - ohne Beschwerden: {(all_states_complaints['complaints'] == 0).sum()}")

# Hover-Text mit überproportional dominantem Thema
hover_text = [
    f"<b>{row['state']}</b><br>"
    f"Beschwerden: {row['complaints']}<br>"
    f"Charakteristisches Thema: {row['topic_label']}<br>"
    f"Überrepräsentationsfaktor: {row['overrepresentation']:.2f}x"
    if row['complaints'] > 0 else
    f"<b>{row['state']}</b><br>Beschwerden: 0<br>Keine Daten"
    for _, row in all_states_complaints.iterrows()
]

# Choroplethenkarte mit überproportional dominantem Thema im Hover
fig = go.Figure(data=go.Choropleth(
    locations=all_states_complaints['state'],
    z=all_states_complaints['complaints'].astype(float),
    text=hover_text,
    locationmode='USA-states',
    colorscale=[
        [0, "#499615"],                                         # keine Beschwerden
        [0.33, "#fbde03"],                                      # durchschnittlich viele Beschwerden
        [1, '#8B0000']                                          # viele Beschwerden
    ],
    colorbar_title="Anzahl Beschwerden",
    hovertemplate='%{text}<extra></extra>'
))

fig.update_layout(
    title_text='Beschwerden nach US-Bundesstaaten (Choroplethenkarte)',
    geo_scope='usa',
    height=600,
    width=1000
)

fig.show()

Datensatz geladen: 5626 Beschwerden
Verfügbare Spalten: ['author', 'posted_on', 'rating', 'text', 'state']

Nationale Themenverteilung:
  Thema 2 (service, comcast, cable): 46.2%
  Thema 4 (comcast, service, account): 22.7%
  Thema 3 (comcast, service, internet): 13.6%
  Thema 0 (called, comcast, said): 11.0%
  Thema 1 (month, comcast, 00): 6.5%

Beschwerdeanzahl + überproportional dominantes Thema pro Bundesstaat:
state  complaints                topic_label  overrepresentation
   FL         775      called, comcast, said            1.112320
   CA         551         month, comcast, 00            1.566547
   GA         413  comcast, service, account            1.118326
   IL         375      called, comcast, said            1.161497
   PA         323 comcast, service, internet            1.208314
   NJ         274      called, comcast, said            1.258465
   TX         274      called, comcast, said            1.357817
   MI         265         month, comcast, 00            1.337

/var/folders/zx/rrh9v2bn61l0xn_h24jrn9fc0000gn/T/ipykernel_9888/429964958.py:48: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# Visualisierung der unstrukturierten Daten
Die Spalte "text" des Datensatzes lässt sich sich durch XXXX visualisieren.